# Context
While working in contract logistics, a customer noticed that the inventory reports sent to them from SAP Business Objects would list partially full locations as completely full. This became especially misleading for our bulk storage locations since these locations were very large and the difference between a nearly empty location and a nearly full location was significant. Because our Business Objects server was managed by a 3rd party with long lead times for creating new reports, I implemented the below tool as a temporary solution and proof of concept. 

# Summary
This report takes as inputs the existing reports that were already set up in our Business Objects server. From these reports, the dimensions of bulk locations, the current quantity and product in each location, the predetermined rules on allowable stacking of each product and the dimensions for each product are stored in a merged table. Next, for each location containing at least 1 item, the maximum quantity of that item that can be stored in that location is calculated, compared to the current quantity in that location and then added to the table. This information is then exported into the recipient's folder as CSV file. Lastly, an optional function is available to report volumetric utilization as a whole. This report can be run using input reports from Business Objects from any date. In this way, information about volume utilization can be tracked and compared over time.

Relevant Columns in the Input Dataset
- Article ID - the unique ID for each different product aka the SKU
- Whse Location - the unique identifier for each storage location in the warehouse, variable quantities may be stored in each location but only 1 SKU
- Avail Width (ft) - the available width in each location to store product
- Avail Depth (ft)  - the available depth in each location to store product
- Avail Height (ft)  - the available height in each location to store product
- Avail Volume (ft3) - the available volume in each location to store product
- Qty in Location - the quantity of a given SKU in a given location

Columns Added to the Final Report
- Max Columns of Item - looking at each location and at the item (if any) that is already being stored in that location, this is the maximum quantity of that item that can be placed within the width of that location if that item is stored in its typical orientation
- Max Rows of Item - looking at each location and at the item (if any) that is already being stored in that location, this is the maximum quantity of that item that can be placed within the length of that location if that item is stored in its typical orientation
- Max Stack - this is the maximum quantity at which a given item is allowed to be stacked verticaly
- Max Item Qty - looking at each location and at the item (if any) that is already being stored in that location, this is the maximum quantity of that item that can be placed within that location if that item is stored in its typical orientation 
- Fillable Item Qty - looking at each location and at the item (if any) that is already being stored in that location, this is the quantity of that item that could be added to that location to fill it to its maximum capacity

In [4]:
from itertools import count
from re import L
import numpy as np
import pandas as pd
import time
import concurrent.futures

LOCATION_DIMENSION_FILE_PATH = 'Bulk Location Dimensions 5-5-22.csv'
PRODUCT_BY_LOCATION_FILE_PATH = 'Product_By_Location_Report 4-22-22.csv'
STACKING_ALLOWANCES_FILE_PATH = 'Stacking Allowances 4-25-22.csv'
ARTICLE_MASTER_FILE_PATH = 'SPWR_ArticleMaster_(KF)_and_PackStructure_(PD)_Report.csv'

# import data
bulk_list_df = pd.DataFrame()
full_df = pd.DataFrame()
stack_df = pd.DataFrame()
article_master_df = pd.DataFrame()

def set_bulk_list_df ():
    global bulk_list_df
    bulk_list_df = pd.read_csv(LOCATION_DIMENSION_FILE_PATH, dtype={'Whse Location': str, 'Article ID': str})
    bulk_list_df = bulk_list_df.drop('Designation', axis=1)
    # numbers come formatted with commas which needs to be cleaned to convert to int for numerical operations
    bulk_list_df = bulk_list_df.replace(',', '', regex=True)
    # remove trailing and leading spaces from applicable data so that identical values between tables 
    # can be recognized
    bulk_list_df['Whse Location'] = bulk_list_df['Whse Location'].str.strip()
    bulk_list_df = bulk_list_df.drop(['Unnamed: 7'], axis=1)
    
def set_full_df ():
    global full_df
    full_df = pd.read_csv(PRODUCT_BY_LOCATION_FILE_PATH, skiprows = 7, dtype={'Whse Location': str, 'Article ID': str})
    # drop empty columns created by cells unmerged in Excel and other unused columns
    full_df = full_df[
    ['Article ID', 'Whse Location']]
    full_df = full_df.replace(',', '', regex=True)
    full_df['Whse Location'] = full_df['Whse Location'].str.strip()
    full_df['Article ID'] = full_df['Article ID'].str.strip()
    
def set_stack_df ():
    global stack_df
    stack_df = pd.read_csv(STACKING_ALLOWANCES_FILE_PATH, skiprows = 1, usecols=['Article ID', 'Max Stack'], 
                           dtype={'Article ID': str, 'Max Stack': int}, nrows=65)
    stack_df = stack_df.replace(',', '', regex=True)
    stack_df['Article ID'] = stack_df['Article ID'].str.strip()
    
def set_article_master_df ():
    global article_master_df
    article_master_df = pd.read_csv(ARTICLE_MASTER_FILE_PATH, dtype={'Article ID': str})
    article_master_df  = article_master_df[
    ['Article ID', 'Article Description', 'PL Length', 'PL Width', 'PL Height', 'PL Volume']]
    article_master_df = article_master_df.rename(columns={'PL Length':'Article Length (ft)', 'PL Width':'Article Width (ft)', 
                                                          'PL Height':'Article Height (ft)','PL Volume':'Article Volume (ft3)'})
    article_master_df = article_master_df.replace(',', '', regex=True)
    article_master_df['Article ID'] = article_master_df['Article ID'].str.strip()

with concurrent.futures.ThreadPoolExecutor() as executor:
    f1 = executor.submit(set_bulk_list_df)
    f2 = executor.submit(set_full_df)
    f3 = executor.submit(set_stack_df)
    f4 = executor.submit(set_article_master_df)
      
# create dataframe bulk_df as subset of full_df containing only bulk locations
bulk_df = full_df
bulk_df = bulk_df.merge(bulk_list_df, how='right', on='Whse Location')

# create column with qty of each item in each location
count_df = bulk_df
count_df['Count'] = 0
count_df = count_df.groupby(['Whse Location']).Count.count().reset_index()
count_df = count_df.drop_duplicates(subset=['Whse Location'])
bulk_df = bulk_df.drop_duplicates(subset=['Whse Location'])
bulk_df = bulk_df.reset_index()
count_df = count_df.reset_index()
bulk_df = bulk_df.merge(count_df, how='left', on='Whse Location')
bulk_df = bulk_df.drop(['index_x', 'Count_x', 'index_y'], axis=1)
bulk_df = bulk_df.rename(columns={'Count_y': 'Qty in Location'})
bulk_df = bulk_df.sort_values('Whse Location')

# add product dimensions
bulk_article_df = article_master_df
bulk_df = bulk_df.merge(bulk_article_df, how='left', on='Article ID')

# create column for maximum number of items that can fit within each location
bulk_df['Max Columns of Item'] = (bulk_df['Avail Width (in)'] / (bulk_df['Article Width (ft)'] * 12)).apply(np.floor)
# Some product is calculated to have zero max columns because it fits one or two inches past the edge of the storage
# lane. However, in actual practice the product is stored in location anyway without issue. The code accounts for this.
bulk_df['Max Columns of Item'] = bulk_df['Max Columns of Item'].replace(0, 1)
bulk_df['Max Rows of Item'] = (bulk_df['Avail Depth  (in)'] / (bulk_df['Article Length (ft)'] * 12)).apply(np.floor)
bulk_df = bulk_df.merge(stack_df, how='left', on='Article ID')

# add column for number of additional items that could be added to each location
bulk_df['Max Item Qty'] = bulk_df['Max Columns of Item'] * bulk_df['Max Rows of Item'] * bulk_df['Max Stack']
bulk_df['Fillable Item Qty'] = bulk_df['Max Item Qty'] - bulk_df['Qty in Location']

# format data frame to be more readable, this is done after all calculations to preserve the accuracy of the original dims
bulk_df[['Article Length (ft)','Article Width (ft)', 'Article Height (ft)', 'Article Volume (ft3)']] = bulk_df[
    ['Article Length (ft)', 'Article Width (ft)', 'Article Height (ft)', 'Article Volume (ft3)']].round(1)
bulk_df[['Avail Width (in)', 'Avail Depth  (in)', 'Avail Height (in)']] = (bulk_df[['Avail Width (in)', 
                                                                        'Avail Depth  (in)', 'Avail Height (in)']] / 12).round(1)
bulk_df = bulk_df.rename(columns={'Avail Width (in)': 'Avail Width (ft)', 'Avail Depth  (in)': 'Avail Depth (ft)',
                                  'Avail Height (in)': 'Avail Height (ft)'})
bulk_df['Avail Volume (in3)'] = (bulk_df['Avail Volume (in3)'] / (12*12*12)).round(1)
bulk_df = bulk_df.rename(columns={'Avail Volume (in3)': 'Avail Volume (ft3)'})
bulk_df

,Article ID,Whse Location,Whse Zone,Avail Width (ft),Avail Depth (ft),Avail Height (ft),Avail Volume (ft3),Qty in Location,Article Description,Article Length (ft),Article Width (ft),Article Height (ft),Article Volume (ft3),Max Columns of Item,Max Rows of Item,Max Stack,Max Item Qty,Fillable Item Qty
0,530039,1A020A,SB1,4.5,35.0,13.3,1955.0,12,PVM SPR-A440-COM-MLSD U 4.2 B1 PV4S,6.7,3.3,4.5,100.4,1.0,5.0,3.0,15.0,3.0
1,530039,1A022A,SB1,4.3,35.0,13.3,1882.6,12,PVM SPR-A440-COM-MLSD U 4.2 B1 PV4S,6.7,3.3,4.5,100.4,1.0,5.0,3.0,15.0,3.0
2,530036,1A024A,SB1,4.7,35.0,13.3,2027.4,12,PVM SPR-A440-COM U 4.2 B1 PV4S 1.5K,6.6,4.2,3.7,100.7,1.0,5.0,3.0,15.0,3.0
3,530036,1A026A,SB1,4.6,35.0,13.3,1991.2,12,PVM SPR-A440-COM U 4.2 B1 PV4S 1.5K,6.6,4.2,3.7,100.7,1.0,5.0,3.0,15.0,3.0
4,530036,1A028A,SB1,4.5,35.0,13.3,1955.0,12,PVM SPR-A440-COM U 4.2 B1 PV4S 1.5K,6.6,4.2,3.7,100.7,1.0,5.0,3.0,15.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,533369,1I115A,SB1,4.2,35.3,13.3,1625.0,56,STORAGE INVERTER SCHNEIDER CONEXT X,2.5,2.0,2.0,10.2,2.0,14.0,4.0,112.0,56.0
708,542928,1I117A,SB1,4.0,35.3,13.3,1560.0,6,ASSEMBLY ESS SUNVAULT BASE PREASSEM,6.0,2.7,4.8,77.9,1.0,5.0,1.0,5.0,-1.0
709,536630,1I119A,SB1,4.2,35.3,13.3,1657.5,6,ASSEMBLY ESS SUNVAULT V1.2 BASE PRE,6.0,2.7,4.8,77.9,1.0,5.0,1.0,5.0,-1.0
710,542928,1I121A,SB1,4.0,35.3,13.3,1560.0,6,ASSEMBLY ESS SUNVAULT BASE PREASSEM,6.0,2.7,4.8,77.9,1.0,5.0,1.0,5.0,-1.0


In [5]:
# optional tool to find volume utilization in bulk area
def find_bulk_utilization():
    bulk_volume_used = (bulk_df['Article Volume (ft3)'] * bulk_df['Qty in Location']).sum()
    bulk_volume_available = bulk_df['Avail Volume (ft3)'].sum()
    bulk_utilization = bulk_volume_used / bulk_volume_available
    print("Bulk Utiliztion", round((100 * bulk_utilization), 2), "%")
    return
find_bulk_utilization()
# export partially filled loc report
# bulk_df.to_csv(r'address', index=False)

Bulk Utiliztion 42.97 %
